## 0.1 Install package and download source code/image.



In [ ]:
%%capture
#@title
!pip install tensorflow_addons

import os
import sys
import tensorflow.compat.v1 as tf

# Download source code.
if "efficientnetv2" not in os.getcwd():
  !git clone --depth 1 https://github.com/google/automl
  os.chdir('automl/efficientnetv2')
  sys.path.append('.')
else:
  !git pull

def download(m):
  if m not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{m}.tgz
    !tar zxf {m}.tgz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

In [ ]:
MODEL = 'efficientnetv2-b0'
import effnetv2_model

# Download checkpoint.
ckpt_path = download(MODEL)
if tf.io.gfile.isdir(ckpt_path):
  ckpt_path = tf.train.latest_checkpoint(ckpt_path)

# Download label map file
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.txt -O labels_map.txt
labels_map = 'labels_map.txt'

# Download images
image_file = 'panda.jpg'
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O {image_file}

In [ ]:
# Build model
tf.keras.backend.clear_session()
model = effnetv2_model.EffNetV2Model(model_name=MODEL)
_ = model(tf.ones([1, 224, 224, 3]), training=False)
model.load_weights(ckpt_path)
cfg = model.cfg

# EfficientNetV2 on CIFAR10.

In [ ]:
!python main_tf2.py --mode=traineval  --model_name=efficientnetv2-b0  --dataset_cfg=cifar10Ft --model_dir={MODEL}_finetune --hparam_str="train.ft_init_ckpt={MODEL},runtime.strategy=gpus,train.batch_size=64"

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Efficienetv2 on Imagenette dataset


In [ ]:
config = {
    'batch_size': 64,
    'epochs': 15
}

In [ ]:
(train_set, test_set), info = tfds.load("imagenette", 
                                           split=["train", "validation"],
                                           as_supervised=True, with_info=True)

class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

def preprocess_img(img, label):
    img = tf.image.resize(img, (224, 224))
    img = img/255
    return img, label

train_set = train_set.map(preprocess_img).batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)
test_set = test_set.map(preprocess_img).batch(config['batch_size'], drop_remainder=True).prefetch(tf.data.AUTOTUNE)

model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  metrics=['accuracy']
)
history = model.fit(train_set, epochs=15, validation_data=test_set)

#Applying Resnet model on  cifar10 dataset

In [ ]:
# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture
base_model = ResNet50V2(include_top=False, input_shape=(32, 32, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Augment the training data with random transformations
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

# Train the model
model.fit(datagen.flow(train_images, train_labels, batch_size=64), epochs=50, validation_data=(test_images, test_labels))

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

#pretrained resnet50 on  cifar10

In [ ]:
mm = keras.applications.ResNet50V2(include_top=False, input_shape=input_shape, weights="imagenet")
out = mm.outputs[0]

nn = keras.layers.GlobalAveragePooling2D(name="avg_pool")(out)
nn = keras.layers.Activation("linear", dtype="float32")(nn)
nn = keras.layers.Dense(num_classes, activation="softmax", name="predictions", dtype="float32")(nn)
resnet50v2_imagenet = keras.models.Model(mm.inputs[0], nn)

resnet50v2_imagenet.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['acc'])
history_resnet50v2_imagenet = resnet50v2_imagenet.fit(train_dataset, epochs=15, validation_data=test_dataset)
with open("history_resnet50v2_imagenet.json", "w") as ff:
    json.dump(history_resnet50v2_imagenet.history, ff)

#EfficientNetV2 on Food101

In [ ]:
import tensorflow_datasets as tfds

# Load the Food101 dataset from TFDS
dataset = tfds.load('food101', split='train[:80%]', as_supervised=True)

# Get the number of classes in the dataset
num_classes = dataset.num_classes

# Define the input shape of the images
input_shape = (224, 224, 3)

In [ ]:
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

def preprocess_img(img, label):
    img = tf.image.resize(img, (224, 224))
    img = img/255
    return img, label

train_set = train_set.map(preprocess_img).batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)
test_set = test_set.map(preprocess_img).batch(config['batch_size'], drop_remainder=True).prefetch(tf.data.AUTOTUNE)

model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.09),
  metrics=['accuracy']
)
history = model.fit(train_set, epochs=15, validation_data=test_set)